### Install and import Dependencies

In [2]:
import cv2 
import mediapipe as mp 

In [3]:
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic

### Make Some Detection

In [ ]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detections
        results = holistic.process(image)

        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
results.face_landmarks.landmark[0]

### Capture Landmarks and Export to CSV

In [6]:
import csv 
import os 
import numpy as np

In [ ]:
num_coords = len(results.pose_landmarks.landmark) + len(results.face_landmarks.landmark)
num_coords

In [7]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [ ]:
landmarks[-1]

In [ ]:
landmarks

In [10]:
with open('coords1.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [11]:
# import os

# # Define the directory where your videos are stored
# directory = r"C:\Users\araya\Downloads\dataset-20240819T164800Z-001\dataset"

# # Loop through all files in the directory
# for filename in os.listdir(directory):
#     # Check if the file is a video by checking its extension
#     if filename.endswith(('.mp4', '.avi', '.mkv', '.mov')):
#         # Print the name of the video file
#         print(filename)


In [12]:
video_path = 'image/กว๊านพะเยา.mp4'

In [13]:
import os
class_name = os.path.splitext(os.path.basename(video_path))[0]

In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pickle

# Initialize Mediapipe Holistic model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Define the directory where your videos are stored
directory = "image/dataset/"

# Dictionary to store keypoints data, with video names as keys
keypoints_data = {}

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file is a video by checking its extension
    if filename.endswith(('.mp4', '.avi', '.mkv', '.mov')):
        print(f"Processing {filename}...")
        video_path = os.path.join(directory, filename)

        class_name = os.path.splitext(os.path.basename(video_path))[0]
        
        cap = cv2.VideoCapture(video_path)
        keypoints_list = []  # Store keypoints for each frame

        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                ret, frame = cap.read()

                if not ret:
                    print("Finished processing frames for this video.")
                    break

                # Recolor Feed
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Make detections
                results = holistic.process(image)

                # Export Coordinates (Pose + Face + Hands)
                try:
                    # Extract Pose landmarks
                    if results.pose_landmarks:
                        pose = results.pose_landmarks.landmark
                        pose_row = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten()
                    else:
                        pose_row = np.zeros(132)  # 33 landmarks * 4 dimensions (x, y, z, visibility)

                    # Extract Face landmarks
                    if results.face_landmarks:
                        face = results.face_landmarks.landmark
                        face_row = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten()
                    else:
                        face_row = np.zeros(468 * 4)  # 468 landmarks * 4 dimensions (x, y, z, visibility)

                    # Extract Left Hand landmarks
                    if results.left_hand_landmarks:
                        left_hand = results.left_hand_landmarks.landmark
                        left_hand_row = np.array([[landmark.x, landmark.y, landmark.z] for landmark in left_hand]).flatten()
                    else:
                        left_hand_row = np.zeros(21 * 3)  # 21 landmarks * 3 dimensions (x, y, z)

                    # Extract Right Hand landmarks
                    if results.right_hand_landmarks:
                        right_hand = results.right_hand_landmarks.landmark
                        right_hand_row = np.array([[landmark.x, landmark.y, landmark.z] for landmark in right_hand]).flatten()
                    else:
                        right_hand_row = np.zeros(21 * 3)  # 21 landmarks * 3 dimensions (x, y, z)

                    # Concatenate all rows (Pose + Face + Left Hand + Right Hand)
                    keypoints_row = np.concatenate([pose_row, face_row, left_hand_row, right_hand_row])

                    # Append the keypoints for this frame to the list
                    keypoints_list.append(keypoints_row)

                except Exception as e:
                    print(f"Error during keypoint extraction: {e}")
                    pass

                # Show the video with landmarks (optional)
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
                mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                
                cv2.imshow('Video Feed', image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        # Release the video capture
        cap.release()
        cv2.destroyAllWindows()

        # Convert the list of keypoints for this video to a Numpy array
        keypoints_data[class_name] = np.array(keypoints_list)

# Save the keypoints data using pickle
with open('keypoints_data.pkl', 'wb') as f:
    pickle.dump(keypoints_data, f)

print("Keypoints extraction and saving completed.")


In [ ]:
print(cap.isOpened())

### Read Data Test

In [ ]:
import pandas as pd 

In [ ]:
df = pd.read_csv("coords1.csv")
df

In [ ]:
df.values[142][0]

In [ ]:
unique_classes = df['class'].unique()
unique_classes

In [ ]:
df.columns[0]

In [ ]:
df.loc[0,"class"]

In [ ]:
X = df.iloc[0,:]

In [ ]:
X = np.array(X)
X

In [ ]:
for i in range(len(df)):
    print(df.loc[i,"class"])
    for j in range(len(df.iloc[i,:])):
        print(j)